In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install praw

You are using pip version 18.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
import praw
import pandas as pd
import datetime as dt

In [7]:
# Creating a reddit crawler object 
reddit = praw.Reddit(client_id='cuDitSOXcwUFJxABTQR93Q',
                     client_secret='BN-dhMP04h9hAQB6ZXzfui4ku7faZQ',
                     user_agent='jamiescrapes',
                     username='JamieRan112',
                     password='Harrogates@@112')

print(reddit.user.me())

JamieRan112


In [8]:
# Creating and crawling a particular subreddit

subreddit = reddit.subreddit('Singapore')
for post in subreddit.search('job insecurity',limit=500):
    print(post.title)
topics_dict = {"title":[],
               "score":[],
               "id":[], "url":[],
               "comms_num": [],
               "created": [],
               "body":[]}
for submission in subreddit.search('job insecurity',limit=100):
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    
covid_data = pd.DataFrame(topics_dict)
def get_date(created):
    return dt.datetime.fromtimestamp(created)
_timestamp = covid_data["created"].apply(get_date)
covid_data = covid_data.assign(timestamp = _timestamp)
covid_data

[Opinion] Thoughts on the past 5 years and well wshes for the future
Saw the amount of support there was on the sexual assault post and wanted to share my own experience with racism.
Dilemma about emigration: why some come back, why some stay?
An ex-PAP supporter's reasons for defecting
Men of r/sg: what are your insecurities?
My reflections on my father
Need some advice rn
Observations by Singaporean(s) at Oxford
Guys, I need serious help...
Should I, as a student without much experience, design a logo for free?


,title,score,id,url,comms_num,created,body,timestamp
0,[Opinion] Thoughts on the past 5 years and wel...,17,hmvqse,https://www.reddit.com/r/singapore/comments/hm...,10,1.594133e+09,The past 5 years have been a rollercoaster. \n...,2020-07-07 22:39:26
1,Saw the amount of support there was on the sex...,1650,n8dr28,https://www.reddit.com/r/singapore/comments/n8...,389,1.620565e+09,"Hello my Singaporean friends.\n\nTo preface, I...",2021-05-09 20:59:23
2,"Dilemma about emigration: why some come back, ...",78,lc33em,https://www.reddit.com/r/singapore/comments/lc...,97,1.612399e+09,"I saw a lot of posts about emigrating or not, ...",2021-02-04 08:43:34
3,An ex-PAP supporter's reasons for defecting,514,hlvl1k,https://www.reddit.com/r/singapore/comments/hl...,152,1.593988e+09,"I used to think PAP was an obvious choice, pur...",2020-07-06 06:25:40
4,Men of r/sg: what are your insecurities?,118,chji7d,https://www.reddit.com/r/singapore/comments/ch...,252,1.564033e+09,I grew up with a ton of insecurities about bei...,2019-07-25 13:29:46
5,My reflections on my father,267,i7kaci,https://www.reddit.com/r/singapore/comments/i7...,58,1.597117e+09,warning: long post of reflections. :)\n\nMy da...,2020-08-11 11:31:21
6,Need some advice rn,0,i7ygmx,https://www.reddit.com/r/singapore/comments/i7...,10,1.597175e+09,Hi im currently 16 retained in sec 3 so i just...,2020-08-12 03:37:01
7,Observations by Singaporean(s) at Oxford,236,8aoby6,https://www.reddit.com/r/singapore/comments/8a...,34,1.523169e+09,#Oxfess16614\n\nThe Ultimate Encyclopaedia of ...,2018-04-08 14:38:17
8,"Guys, I need serious help...",18,2rax2s,https://www.reddit.com/r/singapore/comments/2r...,32,1.420383e+09,"Hey all,\n\nBit a about me. I'm 27, moved to S...",2015-01-04 22:55:18
9,"Should I, as a student without much experience...",8,4g71pe,https://www.reddit.com/r/singapore/comments/4g...,26,1.461477e+09,"Hi graphic designers of /r/singapore, I'm in a...",2016-04-24 13:48:01


# Subreddit Object

We will now create a custom scrape_subreddit object that will help us group all the functions and scrape a subreddit

In [9]:
class scrape_subreddit:
    def __init__(self,reddit,subred,tag,no_of_results):
        self.name = subred
        self.dataframe = None
        self.subreddit = reddit.subreddit(subred)
        self.tag = tag
        self.no_of_results = no_of_results
        self.topics_dict = {"title":[],
               "score":[],
               "id":[], "url":[],
               "comms_num": [],
               "created": [],
               "body":[]}
        
    def scrape_data(self):
        for submission in self.subreddit.search(self.tag,limit=self.no_of_results):
            self.topics_dict["title"].append(submission.title)
            self.topics_dict["score"].append(submission.score)
            self.topics_dict["id"].append(submission.id)
            self.topics_dict["url"].append(submission.url)
            self.topics_dict["comms_num"].append(submission.num_comments)
            self.topics_dict["created"].append(submission.created)
            self.topics_dict["body"].append(submission.selftext)
        return self.topics_dict
    
    def get_dataframe(self):
        self.dataframe = pd.DataFrame(self.scrape_data())
        self.temp_timestamp = self.dataframe["created"].apply(lambda x : dt.datetime.fromtimestamp(x))
        self.dataframe = self.dataframe.assign(timestamp = self.temp_timestamp)
        self.dataframe['year'] = self.dataframe['timestamp'].apply(lambda x : x.year)
        self.dataframe['month'] = self.dataframe['timestamp'].apply(lambda x : x.month)
        self.dataframe['day'] = self.dataframe['timestamp'].apply(lambda x : x.day)
        self.dataframe['time'] = self.dataframe['timestamp'].apply(lambda x : x.time())
        self.dataframe.drop(['timestamp'],axis=1,inplace=True)
        self.dataframe = self.dataframe.drop_duplicates(subset='title',keep="first")
        return self.dataframe
    
    def get_lockdown_data(self):
        self.dataframe = self.get_dataframe()
        self.dataframe = self.dataframe.loc[(self.dataframe['year']==2020) & (self.dataframe['month']>2),:]
        self.dataframe.reset_index(inplace=True)
        return self.dataframe
    
    def get_pre_lockdown_data(self):
        self.dataframe = self.get_dataframe()
        self.dataframe = self.dataframe.loc[(self.dataframe['year']<=2020) & (self.dataframe['month']<3),:]
        self.dataframe.reset_index(inplace=True)
        return self.dataframe
            
                

In [10]:
def scrape(reddit,subreddit,tag,limit):
    reddit = reddit
    scrape_object = scrape_subreddit(reddit,subreddit,tag,limit)
    pre_lockdown_data = scrape_object.get_pre_lockdown_data()
    lockdown_data = scrape_object.get_lockdown_data()
    return pre_lockdown_data,lockdown_data

# Scraping the Subreddits

## 1. r/india

### We'll scrape the subreddit r/india since we only need indian data

### We'll also scrape popular subreddits like r/relationship_advice,r/relationships and r/relationships

## 2. Other subreddits
### Since some subreddits are specific to violence types, we'll scrape them using the keyword 'india'

## r/india

### 1. job insecurity

In [11]:
jobInsecurity_pre_data,jobInsecurity_lockdown_data = scrape(reddit,'Singapore','job insecurity',500)

jobInsecurity_pre_data['keyword'] = ['job insecurity']*len(jobInsecurity_pre_data)
jobInsecurity_lockdown_data['keyword'] = ['job insecurity']*len(jobInsecurity_lockdown_data)
pre_data = jobInsecurity_pre_data
lockdown_data = jobInsecurity_lockdown_data

print(f"Number of data points in jobInsecurity_pre_data i.e, before lockdown : {len(jobInsecurity_pre_data)}")
print(f"Number of data points in jobInsecurity_lockdown_data : {len(jobInsecurity_lockdown_data)}")

Number of data points in jobInsecurity_pre_data i.e, before lockdown : 1
Number of data points in jobInsecurity_lockdown_data : 4


In [13]:
jobInsecurity_pre_data2,jobInsecurity_lockdown_data2 = scrape(reddit,'askSingapore','job insecurity',500)

jobInsecurity_pre_data2['keyword'] = ['job insecurity']*len(jobInsecurity_pre_data2)
jobInsecurity_lockdown_data2['keyword'] = ['job insecurity']*len(jobInsecurity_lockdown_data2)


print(f"Number of data points in jobInsecurity_pre_data2 i.e, before lockdown : {len(jobInsecurity_pre_data2)}")
print(f"Number of data points in jobInsecurity_lockdown_data2 : {len(jobInsecurity_lockdown_data2)}")


# abuse_pre_data2,abuse_lockdown_data2 = scrape(reddit,'relationship_advice','india+abuse',500)

# abuse_pre_data2['violence_type'] = ['abuse']*len(abuse_pre_data2)
# abuse_lockdown_data2['violence_type'] = ['abuse']*len(abuse_lockdown_data2)


# print(f"Number of data points in abuse_pre_data2 i.e, before lockdown : {len(abuse_pre_data2)}")
# print(f"Number of data points in abuse_lockdown_data2 : {len(abuse_lockdown_data2)}")

Number of data points in jobInsecurity_pre_data2 i.e, before lockdown : 0
Number of data points in jobInsecurity_lockdown_data2 : 2


In [14]:
jobInsecurity_pre_data3,jobInsecurity_lockdown_data3 = scrape(reddit,'SingaporeRaw','job insecurity',500)

jobInsecurity_pre_data3['keyword'] = ['job insecurity']*len(jobInsecurity_pre_data3)
jobInsecurity_lockdown_data3['keyword'] = ['job insecurity']*len(jobInsecurity_lockdown_data3)


print(f"Number of data points in jobInsecurity_pre_data3 i.e, before lockdown : {len(jobInsecurity_pre_data3)}")
print(f"Number of data points in jobInsecurity_lockdown_data3 : {len(jobInsecurity_lockdown_data3)}")

# abuse_pre_data3,abuse_lockdown_data3 = scrape(reddit,'relationships','india abuse',500)

# abuse_pre_data3['violence_type'] = ['abuse']*len(abuse_pre_data3)
# abuse_lockdown_data3['violence_type'] = ['abuse']*len(abuse_lockdown_data3)


# print(f"Number of data points in abuse_pre_data3 i.e, before lockdown : {len(abuse_pre_data3)}")
# print(f"Number of data points in abuse_lockdown_data3 : {len(abuse_lockdown_data3)}")

Number of data points in jobInsecurity_pre_data3 i.e, before lockdown : 0
Number of data points in jobInsecurity_lockdown_data3 : 0


In [15]:
jobInsecurity_pre_data4,jobInsecurity_lockdown_data4 = scrape(reddit,'mentalhealthsg','job insecurity',500)

jobInsecurity_pre_data4['keyword'] = ['job insecurity']*len(jobInsecurity_pre_data4)
jobInsecurity_lockdown_data4['keyword'] = ['job insecurity']*len(jobInsecurity_lockdown_data4)


print(f"Number of data points in jobInsecurity_pre_data4 i.e, before lockdown : {len(jobInsecurity_pre_data4)}")
print(f"Number of data points in jobInsecurity_lockdown_data4 : {len(jobInsecurity_lockdown_data4)}")



# abuse_pre_data4,abuse_lockdown_data4 = scrape(reddit,'advice','india abuse',500)

# abuse_pre_data4['violence_type'] = ['abuse']*len(abuse_pre_data4)
# abuse_lockdown_data4['violence_type'] = ['abuse']*len(abuse_lockdown_data4)


# print(f"Number of data points in abuse_pre_data4 i.e, before lockdown : {len(abuse_pre_data4)}")
# print(f"Number of data points in abuse_lockdown_data4 : {len(abuse_lockdown_data4)}")

Number of data points in jobInsecurity_pre_data4 i.e, before lockdown : 0
Number of data points in jobInsecurity_lockdown_data4 : 0


In [16]:
jobInsecurity_pre_data5,jobInsecurity_lockdown_data5 = scrape(reddit,'singaporefi','job insecurity',500)

jobInsecurity_pre_data5['keyword'] = ['job insecurity']*len(jobInsecurity_pre_data5)
jobInsecurity_lockdown_data5['keyword'] = ['job insecurity']*len(jobInsecurity_lockdown_data5)

print(f"Number of data points in jobInsecurity_pre_data5 i.e, before lockdown : {len(jobInsecurity_pre_data5)}")
print(f"Number of data points in jobInsecurity_lockdown_data5 : {len(jobInsecurity_lockdown_data5)}")

RequestException: error with request HTTPSConnectionPool(host='www.reddit.com', port=443): Max retries exceeded with url: /api/v1/access_token (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000125212954F0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

### 2.Dowry

In [16]:
dowry_pre_data,dowry_lockdown_data = scrape(reddit,'india','dowry',500)

dowry_pre_data['violence_type'] = ['dowry']*len(dowry_pre_data)
dowry_lockdown_data['violence_type'] = ['dowry']*len(dowry_lockdown_data)

print(f"Number of data points in dowry_pre_data : {len(dowry_pre_data)}")
print(f"Number of data points in dowry_lockdown_data : {len(dowry_lockdown_data)}")

Number of data points in dowry_pre_data : 30
Number of data points in dowry_lockdown_data : 14


In [17]:
dowry_pre_data2,dowry_lockdown_data2 = scrape(reddit,'relationship_advice','india dowry',500)

dowry_pre_data2['violence_type'] = ['dowry']*len(dowry_pre_data2)
dowry_lockdown_data2['violence_type'] = ['dowry']*len(dowry_lockdown_data2)

print(f"Number of data points in dowry_pre_data2 : {len(dowry_pre_data2)}")
print(f"Number of data points in dowry_lockdown_data2 : {len(dowry_lockdown_data2)}")

Number of data points in dowry_pre_data2 : 0
Number of data points in dowry_lockdown_data2 : 1


In [18]:
dowry_pre_data3,dowry_lockdown_data3 = scrape(reddit,'advice','india dowry',500)

dowry_pre_data3['violence_type'] = ['dowry']*len(dowry_pre_data3)
dowry_lockdown_data3['violence_type'] = ['dowry']*len(dowry_lockdown_data3)

print(f"Number of data points in dowry_pre_data3 : {len(dowry_pre_data3)}")
print(f"Number of data points in dowry_lockdown_data3 : {len(dowry_lockdown_data3)}")

Number of data points in dowry_pre_data3 : 0
Number of data points in dowry_lockdown_data3 : 0


In [35]:
dowry_pre_data4,dowry_lockdown_data4 = scrape(reddit,'relationships','india dowry',500)

dowry_pre_data4['violence_type'] = ['abuse']*len(dowry_pre_data4)
dowry_lockdown_data4['violence_type'] = ['abuse']*len(dowry_lockdown_data4)


print(f"Number of data points in abuse_pre_data2 i.e, before lockdown : {len(dowry_pre_data4)}")
print(f"Number of data points in abuse_lockdown_data2 : {len(dowry_lockdown_data4)}")

Number of data points in abuse_pre_data2 i.e, before lockdown : 1
Number of data points in abuse_lockdown_data2 : 0


### Rape

In [20]:
rape_pre_data,rape_lockdown_data = scrape(reddit,'india','rape',500)

rape_pre_data['violence_type'] = ['rape']*len(rape_pre_data)
rape_lockdown_data['violence_type'] = ['rape']*len(rape_lockdown_data)

print(f"Number of data points in rape_pre_data {len(rape_pre_data)}")
print(f"Number of data points in rape_lockdown_data {len(rape_lockdown_data)}")

Number of data points in rape_pre_data 59
Number of data points in rape_lockdown_data 113


In [21]:
rape_pre_data2,rape_lockdown_data2 = scrape(reddit,'relationship_advice','india rape',500)

rape_pre_data2['violence_type'] = ['rape']*len(rape_pre_data2)
rape_lockdown_data2['violence_type'] = ['rape']*len(rape_lockdown_data2)

print(f"Number of data points in rape_pre_data2 {len(rape_pre_data2)}")
print(f"Number of data points in rape_lockdown_data2 {len(rape_lockdown_data2)}")

Number of data points in rape_pre_data2 2
Number of data points in rape_lockdown_data2 4


### Justice for 

In [22]:
justice_pre_data,justice_lockdown_data = scrape(reddit,'india','justice for',500)

justice_pre_data['violence_type'] = ['justice']*len(justice_pre_data)
justice_lockdown_data['violence_type'] = ['justice'] * len(justice_lockdown_data)

print(f"Number of data points in justice_pre_data {len(justice_pre_data)}")
print(f"Number of data points in justice_lockdown_data {len(justice_lockdown_data)}")

Number of data points in justice_pre_data 45
Number of data points in justice_lockdown_data 35


### Robberies

In [23]:
robbery_pre_data,robbery_lockdown_data = scrape(reddit,'india','robbery',500)

robbery_pre_data['violence_type'] = ['robbery']* len(robbery_pre_data)
robbery_lockdown_data['violence_type'] = ['robbery']*len(robbery_lockdown_data)

print(f"Number of data points in roberry_pre_data {len(robbery_pre_data)}")
print(f"Number of data points in roberry_lockdown_data {len(robbery_lockdown_data)}")

Number of data points in roberry_pre_data 21
Number of data points in roberry_lockdown_data 5


### Harassment

In [24]:
harassment_pre_data,harassment_lockdown_data = scrape(reddit,'india','harassment',1000)

harassment_pre_data['violence_type'] = ['harassment']* len(harassment_pre_data)
harassment_lockdown_data['violence_type'] = ['harassment']*len(harassment_lockdown_data)

print(f"Number of data points in harassment_pre_data {len(harassment_pre_data)}")
print(f"Number of data points in harassment_lockdown_data {len(harassment_lockdown_data)}")

Number of data points in harassment_pre_data 35
Number of data points in harassment_lockdown_data 64


In [25]:
harassment_pre_data2,harassment_lockdown_data2 = scrape(reddit,'relationship_advice','india harassment',1000)

harassment_pre_data2['violence_type'] = ['harassment']* len(harassment_pre_data2)
harassment_lockdown_data2['violence_type'] = ['harassment']*len(harassment_lockdown_data2)

print(f"Number of data points in harassment_pre_data {len(harassment_pre_data2)}")
print(f"Number of data points in harassment_lockdown_data {len(harassment_lockdown_data2)}")

Number of data points in harassment_pre_data 2
Number of data points in harassment_lockdown_data 2


In [26]:
harassment_pre_data3,harassment_lockdown_data3 = scrape(reddit,'relationships','india harassment',1000)

harassment_pre_data3['violence_type'] = ['harassment']* len(harassment_pre_data3)
harassment_lockdown_data3['violence_type'] = ['harassment']*len(harassment_lockdown_data3)

print(f"Number of data points in harassment_pre_data {len(harassment_pre_data3)}")
print(f"Number of data points in harassment_lockdown_data {len(harassment_lockdown_data3)}")

Number of data points in harassment_pre_data 2
Number of data points in harassment_lockdown_data 0


### Survivor

In [27]:
survivor_pre_data,survivor_lockdown_data = scrape(reddit,'india','survivor',1000)

survivor_pre_data['violence_type'] = ['survivor']* len(survivor_pre_data)
survivor_lockdown_data['violence_type'] = ['survivor']*len(survivor_lockdown_data)

print(f"Number of data points in survivor_pre_data {len(survivor_pre_data)}")
print(f"Number of data points in survivor_lockdown_data {len(survivor_lockdown_data)}")

Number of data points in survivor_pre_data 35
Number of data points in survivor_lockdown_data 17


### victim

In [28]:
victim_pre_data,victim_lockdown_data = scrape(reddit,'india','victim',1000)

victim_pre_data['violence_type'] = ['victim']* len(victim_pre_data)
victim_lockdown_data['violence_type'] = ['victim']*len(victim_lockdown_data)

print(f"Number of data points in victim_pre_data {len(victim_pre_data)}")
print(f"Number of data points in victim_lockdown_data {len(victim_lockdown_data)}")

Number of data points in victim_pre_data 40
Number of data points in victim_lockdown_data 105


In [29]:
victim_pre_data2,victim_lockdown_data2 = scrape(reddit,'relationship_advice','india victim',1000)

victim_pre_data2['violence_type'] = ['victim']* len(victim_pre_data2)
victim_lockdown_data2['violence_type'] = ['victim']*len(victim_lockdown_data2)

print(f"Number of data points in victim_pre_data {len(victim_pre_data2)}")
print(f"Number of data points in victim_lockdown_data {len(victim_lockdown_data2)}")

Number of data points in victim_pre_data 2
Number of data points in victim_lockdown_data 0


### Family Abuse

In [30]:
family_abuse_pre_data,family_abuse_lockdown_data = scrape(reddit,'india','family abuse',1000)

family_abuse_pre_data['violence_type'] = ['victim']* len(family_abuse_pre_data)
family_abuse_lockdown_data['violence_type'] = ['victim']*len(family_abuse_lockdown_data)

print(f"Number of data points in victim_pre_data {len(family_abuse_pre_data)}")
print(f"Number of data points in victim_lockdown_data {len(family_abuse_lockdown_data)}")

Number of data points in victim_pre_data 27
Number of data points in victim_lockdown_data 41


In [31]:
family_abuse_pre_data2,family_abuse_lockdown_data2 = scrape(reddit,'relationship_advice','family abuse',1000)

family_abuse_pre_data2['violence_type'] = ['victim']* len(family_abuse_pre_data2)
family_abuse_lockdown_data2['violence_type'] = ['victim']*len(family_abuse_lockdown_data2)

print(f"Number of data points in victim_pre_data {len(family_abuse_pre_data2)}")
print(f"Number of data points in victim_lockdown_data {len(family_abuse_lockdown_data2)}")

Number of data points in victim_pre_data 26
Number of data points in victim_lockdown_data 133


In [32]:
family_abuse_pre_data3,family_abuse_lockdown_data3 = scrape(reddit,'relationships','family abuse',1000)

family_abuse_pre_data3['violence_type'] = ['victim']* len(family_abuse_pre_data3)
family_abuse_lockdown_data3['violence_type'] = ['victim']*len(family_abuse_lockdown_data3)

print(f"Number of data points in victim_pre_data {len(family_abuse_pre_data3)}")
print(f"Number of data points in victim_lockdown_data {len(family_abuse_lockdown_data3)}")

Number of data points in victim_pre_data 22
Number of data points in victim_lockdown_data 112


In [33]:
family_abuse_pre_data4,family_abuse_lockdown_data4 = scrape(reddit,'relationships','family abuse',1000)

family_abuse_pre_data4['violence_type'] = ['victim']* len(family_abuse_pre_data4)
family_abuse_lockdown_data4['violence_type'] = ['victim']*len(family_abuse_lockdown_data4)

print(f"Number of data points in victim_pre_data {len(family_abuse_pre_data4)}")
print(f"Number of data points in victim_lockdown_data {len(family_abuse_lockdown_data4)}")

Number of data points in victim_pre_data 22
Number of data points in victim_lockdown_data 112


# Merging data

In [36]:
pre_data = pd.concat([abuse_pre_data,abuse_pre_data2,abuse_pre_data3,abuse_pre_data4,
                      dowry_pre_data,dowry_pre_data2,dowry_pre_data3,dowry_pre_data4,rape_pre_data,rape_pre_data2,
                      justice_pre_data,robbery_pre_data,harassment_pre_data,harassment_pre_data2,harassment_pre_data3,
                      survivor_pre_data,victim_pre_data, victim_pre_data2,family_abuse_pre_data,family_abuse_pre_data2,
                      family_abuse_pre_data3,family_abuse_pre_data4])

lockdown_data = pd.concat([abuse_lockdown_data,abuse_lockdown_data2,abuse_lockdown_data3,
                           abuse_lockdown_data4,dowry_lockdown_data,dowry_lockdown_data2,
                           dowry_lockdown_data3,dowry_lockdown_data4,rape_lockdown_data,rape_lockdown_data2,
                           justice_lockdown_data,robbery_lockdown_data,harassment_lockdown_data,
                           harassment_lockdown_data2,harassment_lockdown_data3, survivor_lockdown_data,
                           victim_lockdown_data, victim_lockdown_data2,family_abuse_lockdown_data,
                           family_abuse_lockdown_data2,family_abuse_lockdown_data3,family_abuse_lockdown_data4])

In [37]:
print(f"Length of lockdown data {len(lockdown_data)}")
lockdown_data = lockdown_data.drop_duplicates(subset='title',keep='first')
print(f"Length of unique lockdown data {len(lockdown_data)}")

Length of lockdown data 930
Length of unique lockdown data 721


In [38]:
print(f"Length of pre lockdown data {len(pre_data)}")
pre_data = pre_data.drop_duplicates(subset='title',keep='first')
print(f"Length of unique pre lockdown data {len(pre_data)}")

Length of pre lockdown data 439
Length of unique pre lockdown data 395


## 2. Other subreddits 

We'll focus on the following subreddits

1. raisedbynarcissists
2. adultsurvivors
3. legaladvice
4. Parenting
5. MensRights
6. abuse
7. GenderCritical

In [39]:
multiple_subs_pre,multiple_subs_lockdown = scrape(reddit,'raisebynarcissists+adultsurvivors+legaladvice+Parenting+MensRights+abuse+GenderCritical+lgbt'
                                                  ,'india',3000)
print(len(multiple_subs_pre))
print(len(multiple_subs_lockdown))

41
139


In [40]:
pre_data = pd.concat([pre_data,multiple_subs_pre])
lockdown_data = pd.concat([lockdown_data,multiple_subs_lockdown])

In [41]:
len(lockdown_data)

860

In [42]:
lockdown_data.to_csv('lockdown_reddit.csv')
pre_data.to_csv('pre_lockdown_reddit.csv')

# Modelling

We'll extract the following features from the data also while filtering irrelevant entries parallely

* Age
* Gender
* Sexual Orientation
* Imagery
* Location
* Language
* Time of day when active
* relationship between victim and abuser


In [43]:
# Let's see what do we have

df = pd.read_csv('lockdown_reddit.csv')

In [44]:
print(df.dtypes)
df.head()

Unnamed: 0         int64
index              int64
title             object
score            float64
id                object
url               object
comms_num        float64
created          float64
body              object
year             float64
month            float64
day              float64
time              object
violence_type     object
dtype: object


,Unnamed: 0,index,title,score,id,url,comms_num,created,body,year,month,day,time,violence_type
0,0,0,There is NO ROOM for abuse.Period. Digital Pai...,3136.0,gfoc94,https://i.redd.it/xfvcce15rhx41.jpg,328.0,1.588951e+09,NaN,2020.0,5.0,8.0,20:57:12,abuse
1,1,1,Capitalism and Private firms during covid. Wor...,281.0,gsmest,https://www.reddit.com/r/india/comments/gsmest...,86.0,1.590757e+09,My friend works from 8:30 to 17:30. Meanwhile ...,2020.0,5.0,29.0,18:47:13,abuse
2,2,2,"Behind closed doors, the biggest viruses are m...",1129.0,fzzzji,https://i.redd.it/dc1v110v0fs41.jpg,172.0,1.586738e+09,NaN,2020.0,4.0,13.0,06:25:34,abuse
3,3,3,"Bait for one, Death for another. But is it jus...",56.0,gx7zv5,https://i.redd.it/dv07s7qsd4351.jpg,15.0,1.591404e+09,NaN,2020.0,6.0,6.0,06:24:05,abuse
4,4,4,"Lawyers of India, if I try to file a domestic ...",45.0,gvzj3d,https://www.reddit.com/r/india/comments/gvzj3d...,16.0,1.591238e+09,My ex-boyfriend (2 year relationship) is from ...,2020.0,6.0,4.0,08:14:50,abuse


### Age

In [45]:
df['body'] = df['body'].fillna('None').apply(str)

In [46]:
import re
def age_extractor(sentence):
    match = re.findall(r"\b\d{2}(?!%)[^\d]", sentence)

    if match:
        return re.findall(r"\b\d{2}(?!%)[^\d]", sentence)[0][:2]
    else:
        return None


In [47]:
df['age'] = df['body'].apply(age_extractor)

In [48]:
df['age']

0      None
1        30
2      None
3      None
4      None
       ... 
855    None
856      18
857    None
858      16
859      21
Name: age, Length: 860, dtype: object

In [49]:
df.drop(['city','state'],axis=1,inplace=True)

KeyError: "['city' 'state'] not found in axis"

In [50]:
df = df.loc[df['title'].notna(),:]

In [51]:
len(df)

860

In [52]:
df

,Unnamed: 0,index,title,score,id,url,comms_num,created,body,year,month,day,time,violence_type,age
0,0,0,There is NO ROOM for abuse.Period. Digital Pai...,3136.0,gfoc94,https://i.redd.it/xfvcce15rhx41.jpg,328.0,1.588951e+09,None,2020.0,5.0,8.0,20:57:12,abuse,None
1,1,1,Capitalism and Private firms during covid. Wor...,281.0,gsmest,https://www.reddit.com/r/india/comments/gsmest...,86.0,1.590757e+09,My friend works from 8:30 to 17:30. Meanwhile ...,2020.0,5.0,29.0,18:47:13,abuse,30
2,2,2,"Behind closed doors, the biggest viruses are m...",1129.0,fzzzji,https://i.redd.it/dc1v110v0fs41.jpg,172.0,1.586738e+09,None,2020.0,4.0,13.0,06:25:34,abuse,None
3,3,3,"Bait for one, Death for another. But is it jus...",56.0,gx7zv5,https://i.redd.it/dv07s7qsd4351.jpg,15.0,1.591404e+09,None,2020.0,6.0,6.0,06:24:05,abuse,None
4,4,4,"Lawyers of India, if I try to file a domestic ...",45.0,gvzj3d,https://www.reddit.com/r/india/comments/gvzj3d...,16.0,1.591238e+09,My ex-boyfriend (2 year relationship) is from ...,2020.0,6.0,4.0,08:14:50,abuse,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,134,231,Is it possible to amend the governing law loca...,2.0,ges6gh,https://www.reddit.com/r/legaladvice/comments/...,3.0,1.588827e+09,I'm located in USA and have a LLC. The manufac...,2020.0,5.0,7.0,10:30:01,NaN,None
856,135,232,Instagram Sex Service,0.0,g5p6ot,https://www.reddit.com/r/legaladvice/comments/...,11.0,1.587537e+09,"I stay in India. So today, I matched with this...",2020.0,4.0,22.0,12:12:53,NaN,18
857,136,234,GET TWEETIG AND EMAILING!! It gets worse - men...,9.0,g4eqtv,https://www.reddit.com/r/MensRights/comments/g...,4.0,1.587357e+09,Some have seen this posted. I say it gets wors...,2020.0,4.0,20.0,10:10:28,NaN,None
858,137,235,Paid extra for 3 years for Netflix.,0.0,g6wnlm,https://www.reddit.com/r/legaladvice/comments/...,9.0,1.587711e+09,Hi everyone. A stupid post alert. \n\nApparent...,2020.0,4.0,24.0,12:26:59,NaN,16


In [53]:
df.drop(['Unnamed: 0','index','Unnamed: 14'],axis=1,inplace=True)

KeyError: "['Unnamed: 14'] not found in axis"

In [54]:
df

,Unnamed: 0,index,title,score,id,url,comms_num,created,body,year,month,day,time,violence_type,age
0,0,0,There is NO ROOM for abuse.Period. Digital Pai...,3136.0,gfoc94,https://i.redd.it/xfvcce15rhx41.jpg,328.0,1.588951e+09,None,2020.0,5.0,8.0,20:57:12,abuse,None
1,1,1,Capitalism and Private firms during covid. Wor...,281.0,gsmest,https://www.reddit.com/r/india/comments/gsmest...,86.0,1.590757e+09,My friend works from 8:30 to 17:30. Meanwhile ...,2020.0,5.0,29.0,18:47:13,abuse,30
2,2,2,"Behind closed doors, the biggest viruses are m...",1129.0,fzzzji,https://i.redd.it/dc1v110v0fs41.jpg,172.0,1.586738e+09,None,2020.0,4.0,13.0,06:25:34,abuse,None
3,3,3,"Bait for one, Death for another. But is it jus...",56.0,gx7zv5,https://i.redd.it/dv07s7qsd4351.jpg,15.0,1.591404e+09,None,2020.0,6.0,6.0,06:24:05,abuse,None
4,4,4,"Lawyers of India, if I try to file a domestic ...",45.0,gvzj3d,https://www.reddit.com/r/india/comments/gvzj3d...,16.0,1.591238e+09,My ex-boyfriend (2 year relationship) is from ...,2020.0,6.0,4.0,08:14:50,abuse,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,134,231,Is it possible to amend the governing law loca...,2.0,ges6gh,https://www.reddit.com/r/legaladvice/comments/...,3.0,1.588827e+09,I'm located in USA and have a LLC. The manufac...,2020.0,5.0,7.0,10:30:01,NaN,None
856,135,232,Instagram Sex Service,0.0,g5p6ot,https://www.reddit.com/r/legaladvice/comments/...,11.0,1.587537e+09,"I stay in India. So today, I matched with this...",2020.0,4.0,22.0,12:12:53,NaN,18
857,136,234,GET TWEETIG AND EMAILING!! It gets worse - men...,9.0,g4eqtv,https://www.reddit.com/r/MensRights/comments/g...,4.0,1.587357e+09,Some have seen this posted. I say it gets wors...,2020.0,4.0,20.0,10:10:28,NaN,None
858,137,235,Paid extra for 3 years for Netflix.,0.0,g6wnlm,https://www.reddit.com/r/legaladvice/comments/...,9.0,1.587711e+09,Hi everyone. A stupid post alert. \n\nApparent...,2020.0,4.0,24.0,12:26:59,NaN,16


In [55]:
df.to_csv('new_reddit_domestic_violence.csv')